# Code for gathering the NAATBatt applicable facilities and mapping census demographic data with them

__________________________________________________________________________________________________________

In [2]:
import pandas as pd
import numpy as np
import statistics as s
import math as m
from scipy.stats import ttest_ind
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import random
plt.rcParams["font.family"] = "Arial"
plt.rcParams['figure.dpi']=600

## Import Data

Universal Stats

In [ ]:
# 3.5 MWh and less than 60,000 kg; 3500 kWh and less than 60,000 kg; https://one.ai/products/aries-grid; ONE
# 2.982 MWh and 38,000 kg ; 2982 kWh and 38,000 kg; https://6581523.fs1.hubspotusercontent-na1.net/hubfs/6581523/B-VAULT_Datasheet.pdf?hsCtaTracking=babb0809-32bd-4eb5-982e-e607c52a634e%7C53e9bc72-40f9-4934-bad8-22f3504162f2; Energy Vault
# 1.65 MWh, 2 MWh, 2.5 MWh and 20,000, 22,680, and 25,400 kg; https://7828151.fs1.hubspotusercontent-na1.net/hubfs/7828151/Brochure%20et%20spec%20sheet%202022/EVLOFLEX_Tech_Sheet_EN.pdf?utm_medium=email&_hsmi=258711713&_hsenc=p2ANqtz-9SuPY6DIgbeYsKcjhL7mx0IBHxRoBWbnWQgUWHZDk0W2XTtZ0r8v7jHvJF38bMDfA8YDkqlQzb4Lo0UGy5PyC90Lu5bQ&utm_content=258711713&utm_source=hs_automation; https://7828151.fs1.hubspotusercontent-na1.net/hubfs/7828151/Brochure%20et%20spec%20sheet%202022/EVLOFLEX_Tech_Sheet_EN.pdf?utm_medium=email&_hsmi=258711713&_hsenc=p2ANqtz-9SuPY6DIgbeYsKcjhL7mx0IBHxRoBWbnWQgUWHZDk0W2XTtZ0r8v7jHvJF38bMDfA8YDkqlQzb4Lo0UGy5PyC90Lu5bQ&utm_content=258711713&utm_source=hs_automation; EVLO Energy Storage
# 3.854 MWh, 3.916 MWh and 30,500, and 38,100 kg; https://www.tesla.com/sites/default/files/downloads/Lithium-Ion_Battery_Emergency_Response_Guide_en.pdf; https://www.tesla.com/megapack; TESLA
# All using LFP chemistry

spec1 = 38000/2982
spec2 = 25400/2500
spec3 = 22680/2000
spec4 = 20000/1650
spec5 = 38100/3916
spec6 = 30500/3854

specs = [spec1,spec2,spec3,spec4,spec5,spec6]
spec_avg = sum(specs)/6

### US LCIA data

In [ ]:
# list creation to import all scenarios
dfs = []
scenarios = range(0,24,1)

for x in scenarios:
    df = pd.read_excel('Data/Results/BSS_LFP_'+str(x)+'.xlsx',sheet_name="Impacts",skiprows=1)
    dfs.append(df)

# all scenarios, clean columns
scenarios = pd.read_csv('Data/scenarios_final.csv')
scenarios.drop(columns=["recycle","lat6","lon6","trans4","transdist (m)","cradel2gate_transdist","scenarioID","Unnamed: 0"],inplace=True)
scenarios.drop_duplicates(inplace=True)
scenarios.reset_index(inplace=True,drop=True)

# 12 brine
# 6 spodumene
# 6 geothermal brine

contbrine_scen = dfs[0:12]
spod_scen = dfs[12:18]
geobrine_scen = dfs[18:24]

diff_scens = [contbrine_scen, spod_scen, geobrine_scen]

## US Analysis

### LCIA stats for all 18 impact categories and all scenarios

In [ ]:
# get stats median, mean, min, max, and standard deviation per impact category using all scenarios
impacts = []
for x in dfs:
    list_impact = x['Result'].to_list()
    impacts.append(list_impact)

df_impacts = pd.DataFrame(impacts)
medians = df_impacts.median(axis=0).to_list()
means = df_impacts.mean(axis=0).to_list()
mins = df_impacts.min(axis=0).to_list()
maxes = df_impacts.max(axis=0).to_list()
stdevs = df_impacts.std(axis=0).to_list()
impact_names = dfs[0]['Impact category'].to_list()
impact_units = dfs[0]['Reference unit'].to_list()

df_impact_stats = pd.DataFrame([impact_names, impact_units, medians, means, mins, maxes, stdevs])

df_impact_stats = df_impact_stats.transpose()

df_impact_stats.rename(columns={0:'Impacts',1:'Units',2:'median',3:'mean',4:'minimum',5:'maximum',6:'standard deviation'},inplace=True)
df_impact_stats['Lower bound pct'] = (df_impact_stats['minimum']-df_impact_stats['mean'])/df_impact_stats['mean']*100
df_impact_stats['Upper bound pct'] = (df_impact_stats['maximum']-df_impact_stats['mean'])/df_impact_stats['mean']*100


# difference influenced by lithium type (spodumene, continental brine or geothermal brine), location of facilities/transportation
df_impact_stats.to_excel("Data/Results/Scenario_LCA_Impact_Stats.xlsx")


### LCIA stats for all 18 impact categories by different lithium sources (continental brine, geothermal brine, and spodumene)

In [ ]:
# get stats median, mean, min, max, and standard deviation per impact category for different lithium sources
df_diff_scenarios_impacts = []
df_diff_scen_all_vals = []

for i in diff_scens:
    impacts = []
    for x in i:
        list_impact = x['Result'].to_list()
        impacts.append(list_impact)

    df_impacts = pd.DataFrame(impacts)
    df_diff_scen_all_vals.append(df_impacts)
    medians = df_impacts.median(axis=0).to_list()
    means = df_impacts.mean(axis=0).to_list()
    mins = df_impacts.min(axis=0).to_list()
    maxes = df_impacts.max(axis=0).to_list()
    impact_names = dfs[0]['Impact category'].to_list()
    impact_units = dfs[0]['Reference unit'].to_list()

    df_impact_stats_scenarios = pd.DataFrame([impact_names, impact_units, medians, means, mins, maxes])

    df_impact_stats_scenarios = df_impact_stats_scenarios.transpose()

    df_impact_stats_scenarios.rename(columns={0:'Impacts',1:'Units',2:'median',3:'mean',4:'minimum',5:'maximum'},inplace=True)
    df_diff_scenarios_impacts.append(df_impact_stats_scenarios)

# df_diff_scenarios_impacts[0]
# df_diff_scenarios_impacts[1]
# df_diff_scenarios_impacts[2]

ternary diagram for different lithium sources

In [ ]:
li_source_names = ["cont_brine","spod","geo_brine"]

with pd.ExcelWriter('Data/Results/Scenario_LCA_Impact_Stats_Diff_Lithium.xlsx') as writer:  # doctest: +SKIP
    for x in range(0,3,1):
        df_diff_scenarios_impacts[x].to_excel(writer, sheet_name=li_source_names[x])

# clean data to fit plotly.express calls below
df_diff_li_source_results = [df_diff_scenarios_impacts[0]['mean'],df_diff_scenarios_impacts[1]['mean'],df_diff_scenarios_impacts[2]['mean']]
index_labels = ['ContBrine', 'Spodumene', 'GeoBrine']
df_diff_li_source_results = pd.DataFrame(df_diff_li_source_results,index=index_labels)
df_diff_li_source_results = df_diff_li_source_results.transpose()

import plotly.express as px
fig = px.scatter_ternary(df_diff_li_source_results, a="ContBrine", b="Spodumene", c="GeoBrine")
fig.show()
fig.write_image('Data/Results/Figures/ternary_diagram.png')


### T-test for impact catagories for different resources

In [ ]:
ttest_results1 = [] # cont and geothermal
ttest_results2 = [] # cont and spodumene
ttest_results3 = [] # spodumene and geothermal

for x in df_diff_scen_all_vals[0].columns:
    pval = ttest_ind(df_diff_scen_all_vals[0][x].to_list(),df_diff_scen_all_vals[2][x].to_list())[1]
    ttest_results1.append(pval)

for x in df_diff_scen_all_vals[0].columns:
    pval = ttest_ind(df_diff_scen_all_vals[0][x].to_list(),df_diff_scen_all_vals[1][x].to_list())[1]
    ttest_results2.append(pval)

for x in df_diff_scen_all_vals[1].columns:
    pval = ttest_ind(df_diff_scen_all_vals[1][x].to_list(),df_diff_scen_all_vals[2][x].to_list())[1]
    ttest_results3.append(pval)

pval_diff_li = pd.DataFrame(zip(*[impact_names,ttest_results1,ttest_results2,ttest_results3]))
pval_diff_li.rename(columns={0:'Impacts',1:'Cont-Geo Brine',2:'Cont-Spodumene',3:'Geo-Spodumene'},inplace=True)


from scipy.stats import f_oneway
from scipy.stats import tukey_hsd

anova_results = []
for x in range(0,18,1):
    anova_results.append(f_oneway(df_diff_scen_all_vals[0][x].to_list(),df_diff_scen_all_vals[1][x].to_list(),df_diff_scen_all_vals[2][x].to_list())[1])

# anova_tukey_results = []
# anova_tukey_comps = ["Cont Brine - Spodumene","Cont Brine - Geo Brine","Spodumene - Cont Brine","Spodumene - Geo Brine","Geo Brine - Cont Brine","Geo Brine - Spodumene"]
# for x in range(0,18,1):
#     res = tukey_hsd(df_diff_scen_all_vals[0][x].to_list(),df_diff_scen_all_vals[1][x].to_list(),df_diff_scen_all_vals[2][x].to_list())
#     df = pd.DataFrame([anova_tukey_comps,res.pvalue])
#     anova_tukey_results.append(df)

for x in range(0,18,1):
    res = tukey_hsd(df_diff_scen_all_vals[0][x].to_list(),df_diff_scen_all_vals[1][x].to_list(),df_diff_scen_all_vals[2][x].to_list())
    print(res)

### Contribution trees for selected impact categories (global warming potential, freshwater ecotoxicity, terrestrial ecotoxicity, particulate matter formation, and human carcinogenic potential)

In [ ]:
steps_manuf = [
    'BSS LFP',
    'LFP module',
    'LFP cell',
    'Cathode',
    'Cathode paste',
    'Lithium iron phosphate powder',
    'Lithium hydroxide',
    'Anode',
    'Anode paste',
    'Coated spherical graphite',
    'Electrolyte',
    'Separator',
    'Cell container',
    'Housing',
    'System controller',
    'Inverter',
    'Charger'
]

scenarios = range(0,24,1)

steps_names_dict = {}

for x in steps_manuf:
    steps_names_dict.update({steps_manuf.index(x):x})


Particulate matter 2.5

In [ ]:
dfs_PM = []

for x in scenarios:
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_PM.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_PM.append(df)


process_contributions_PM = []
for x in scenarios:
    df = dfs_PM[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg PM2.5 eq]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg PM2.5 eq]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_PM.append(scenario_process_contributions)


Global warming potential

In [ ]:
dfs_GWP = []

for x in scenarios:
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_GWP.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_GWP.append(df)


process_contributions_GWP = []
for x in scenarios:
    df = dfs_GWP[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg CO2 eq]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg CO2 eq]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_GWP.append(scenario_process_contributions)



Freshwater ecotoxicity

In [ ]:
dfs_FET = []

for x in scenarios:
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_FET.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_FET.append(df)


process_contributions_FET = []
for x in scenarios:
    df = dfs_FET[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_FET.append(scenario_process_contributions)


Human carcinogen toxicity

In [ ]:
dfs_HCT = []

for x in scenarios:
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_HTP.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_HCT.append(df)


process_contributions_HCT = []
for x in scenarios:
    df = dfs_HCT[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_HCT.append(scenario_process_contributions)




Terrestrial ecotoxicity

In [ ]:
dfs_TET = []

for x in scenarios:
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_TET.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_TET.append(df)


process_contributions_TET = []
for x in scenarios:
    df = dfs_TET[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_TET.append(scenario_process_contributions)


get average of process contributions for selected impact categories for all scenarios

In [ ]:
diff_contributions = [process_contributions_FET,process_contributions_GWP,process_contributions_HCT,process_contributions_PM,process_contributions_TET]

lists_scenario_contributions = []
for x in diff_contributions:
    df = pd.DataFrame(x)
    df.rename(columns=steps_names_dict,inplace=True)
    diff_cont_scenario_averages = df.mean(axis=0).to_list()
    lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions = df_scenario_contributions.transpose()

df_scenario_contributions.rename(columns={0:'Freshwater Ecotoxicity',1:'Global Warming Potential',2:'Human Carcinogen Toxicity',3:'Particulate Matter 2.5',4:'Terrestrial Ecotoxicity'},inplace=True)

df_scenario_contributions["Production Steps"] = steps_manuf
df_scenario_contributions.to_csv("Data/Results/Scenario_Process_Contributions_Perc.csv")


get average of process contributions for selected impact categories for scenarios by lithium source

In [ ]:
# continental brine
lists_scenario_contributions = []
for x in diff_contributions:
    df = pd.DataFrame(x[0:12])
    df.rename(columns=steps_names_dict,inplace=True)
    diff_cont_scenario_averages = df.mean(axis=0).to_list()
    lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions_contbrine = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions_contbrine = df_scenario_contributions_contbrine.transpose()

df_scenario_contributions_contbrine.rename(columns={0:'Freshwater Ecotoxicity',1:'Global Warming Potential',2:'Human Carcinogen Toxicity',3:'Particulate Matter 2.5',4:'Terrestrial Ecotoxicity'},inplace=True)

df_scenario_contributions_contbrine["Production Steps"] = steps_manuf
df_scenario_contributions_contbrine.to_excel("Data/Results/Scenario_Process_Contributions_Perc_contbrine.xlsx",sheet_name="contbrine")

# geothermal brine
lists_scenario_contributions = []
for x in diff_contributions:
    df = pd.DataFrame(x[12:18])
    df.rename(columns=steps_names_dict,inplace=True)
    diff_cont_scenario_averages = df.mean(axis=0).to_list()
    lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions_geobrine = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions_geobrine = df_scenario_contributions_geobrine.transpose()

df_scenario_contributions_geobrine.rename(columns={0:'Freshwater Ecotoxicity',1:'Global Warming Potential',2:'Human Carcinogen Toxicity',3:'Particulate Matter 2.5',4:'Terrestrial Ecotoxicity'},inplace=True)

df_scenario_contributions_geobrine["Production Steps"] = steps_manuf
df_scenario_contributions_geobrine.to_excel("Data/Results/Scenario_Process_Contributions_Perc_geobrine.xlsx",sheet_name="geobrine")

# spodumene
lists_scenario_contributions = []
for x in diff_contributions:
    df = pd.DataFrame(x[18:24])
    df.rename(columns=steps_names_dict,inplace=True)
    diff_cont_scenario_averages = df.mean(axis=0).to_list()
    lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions_spod = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions_spod = df_scenario_contributions_spod.transpose()

df_scenario_contributions_spod.rename(columns={0:'Freshwater Ecotoxicity',1:'Global Warming Potential',2:'Human Carcinogen Toxicity',3:'Particulate Matter 2.5',4:'Terrestrial Ecotoxicity'},inplace=True)

df_scenario_contributions_spod["Production Steps"] = steps_manuf
df_scenario_contributions_spod.to_excel("Data/Results/Scenario_Process_Contributions_Perc_spod.xlsx",sheet_name="spod")


df_scenario_contributions_lisource = [df_scenario_contributions_contbrine,df_scenario_contributions_geobrine,df_scenario_contributions_spod]


### Process contributing most to impacts


In [ ]:

dfs = []

# clean data, get process level impact contributions for five impact categories
for x in scenarios:
    df = pd.read_excel('Data/Results/BSS_LFP_'+str(x)+'.xlsx',sheet_name="Process impact contributions",skiprows=2)
    df = df.loc[2:,]
    df.drop(columns = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 3", "Process"],inplace=True)
    df.rename(columns={'Unnamed: 2':'Impact Category'},inplace=True)
    df = df.loc[((df['Impact Category'] == 'Fine particulate matter formation') | (df['Impact Category'] == 'Freshwater ecotoxicity') | (df['Impact Category'] == 'Global warming') | (df['Impact Category'] == 'Human carcinogenic toxicity') | (df['Impact Category'] == 'Terrestrial ecotoxicity') | (df['Impact Category'] == 'Mineral resource scarcity')),]
    df.set_index(df.columns[0],inplace=True)
    df = df.transpose()
    dfs.append(df)

max_process_all_scenarios = []

# get max impact contribution and the process name
for x in dfs:
    max_process = []
    for i in x.columns:
        maxVal = pd.to_numeric(x[i]).idxmax(skipna=True)
        max_process.append(maxVal)
    max_process_all_scenarios.append(max_process)


df_max_process_scenarios = pd.DataFrame(columns = ["PM","Freshwater Ecotoxicity","GWP","Human Carcinogen Toxicity","Terrestrial Ecotoxicity","Mineral Resource Scarcity"])

for x in max_process_all_scenarios:
    df_max_process_scenarios.loc[len(df_max_process_scenarios)] = x


### Average impact of electricity use for all scenarios
scope 2 electricity use; electricity used in the process, exculding electricity used to produce the materials being used 


In [ ]:
dfs = []

# clean data
for x in scenarios:
    df = pd.read_excel('Data/Results/BSS_LFP_'+str(x)+'.xlsx',sheet_name="Process impact contributions",skiprows=2)
    df = df.loc[2:,]
    df.drop(columns = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 3", "Process"],inplace=True)
    df.rename(columns={'Unnamed: 2':'Impact Category'},inplace=True)
    df.set_index(df.columns[0],inplace=True)
    df = df.transpose()
    dfs.append(df)

# get all electricity processes summed (NOT ENERGY, ELECTRICITY)
dfs_electricity_impacts = []
for x in dfs:
    x.reset_index(inplace=True)
    x.rename(columns={'index':'Impacts'},inplace=True)
    x = x[x['Impacts'].str.contains('electricity')]
    x.drop(columns=['Impacts'],inplace=True)
    dfs_electricity_impacts.append(pd.DataFrame(x.sum().copy()))

dfs_electricity_impacts = pd.concat(dfs_electricity_impacts, axis=1, ignore_index=False)
dfs_electricity_impacts = pd.DataFrame(dfs_electricity_impacts.mean(axis=1))
dfs_electricity_impacts['scenario_avg_total_impact'] = df_impact_stats['mean'].to_list()
dfs_electricity_impacts['ratio_electricity_all_impacts'] = dfs_electricity_impacts[0]/dfs_electricity_impacts['scenario_avg_total_impact']*100



### Average impact of transportation for all scenarios
transportation used in the process, exculding electricity used to produce the materials being used 

In [ ]:
dfs = []

# clean data
for x in scenarios:
    df = pd.read_excel('Data/Results/BSS_LFP_'+str(x)+'.xlsx',sheet_name="Process impact contributions",skiprows=2)
    df = df.loc[2:,]
    df.drop(columns = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 3", "Process"],inplace=True)
    df.rename(columns={'Unnamed: 2':'Impact Category'},inplace=True)
    df.set_index(df.columns[0],inplace=True)
    df = df.transpose()
    dfs.append(df)

# get all transportation processes summed
dfs_transport_impacts = []
for x in dfs:
    x.reset_index(inplace=True)
    x.rename(columns={'index':'Impacts'},inplace=True)
    x = x[x['Impacts'].str.contains('transport')]
    x.drop(columns=['Impacts'],inplace=True)
    dfs_transport_impacts.append(pd.DataFrame(x.sum().copy()))

dfs_transport_impacts = pd.concat(dfs_transport_impacts, axis=1, ignore_index=False)
dfs_transport_impacts = pd.DataFrame(dfs_transport_impacts.mean(axis=1))
dfs_transport_impacts['scenario_avg_total_impact'] = df_impact_stats['median'].to_list()
dfs_transport_impacts['ratio_transport_all_impacts'] = dfs_transport_impacts[0]/dfs_transport_impacts['scenario_avg_total_impact']*100

## US Figures

#### Sunburst pie chart, system level

In [ ]:
df_scenario_contributions_PC = df_scenario_contributions.copy()
df_scenario_contributions_PC['Freshwater Ecotoxicity'] = df_scenario_contributions_PC['Freshwater Ecotoxicity']*100
df_scenario_contributions_PC['Global Warming Potential'] = df_scenario_contributions_PC['Global Warming Potential']*100
df_scenario_contributions_PC['Human Carcinogen Toxicity'] = df_scenario_contributions_PC['Human Carcinogen Toxicity']*100
df_scenario_contributions_PC['Particulate Matter 2.5'] = df_scenario_contributions_PC['Particulate Matter 2.5']*100
df_scenario_contributions_PC['Terrestrial Ecotoxicity'] = df_scenario_contributions_PC['Terrestrial Ecotoxicity']*100
df_scenario_contributions_PC = df_scenario_contributions_PC.round(2)
df_scenario_contributions_PC.set_index(['Production Steps'],inplace=True)

color_sequence = ['','#ffbe0b','#fb5607','#ff006e','#8338ec','#3a86ff','#FFD258']
for x in df_scenario_contributions_PC.columns:

    fig = go.Figure(go.Sunburst(
        labels=['LFP Battery System','Module','Housing','System controller','Inverter','Charger','Cell'],
        parents=['','LFP Battery System',"LFP Battery System","LFP Battery System","LFP Battery System","LFP Battery System","Module"],
        values=[df_scenario_contributions_PC.at['BSS LFP',x],
                df_scenario_contributions_PC.at['LFP module',x],
                df_scenario_contributions_PC.at['Housing',x],
                df_scenario_contributions_PC.at['System controller',x],
                df_scenario_contributions_PC.at['Inverter',x],
                df_scenario_contributions_PC.at['Charger',x],
                df_scenario_contributions_PC.at['LFP cell',x],
        ],
        branchvalues="total",
        marker=dict(colors=color_sequence)
    ))
    fig.update_traces(textinfo="percent parent")
    #fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),uniformtext=dict(minsize=15, mode='hide'))

    fig.show()
    fig.write_image("Data/Results/Figures/scenario_avg_sunburst_"+x+".png",format='png')


#### Suburst chart, cell level

In [ ]:
df_scenario_contributions_PC = df_scenario_contributions.copy()
df_scenario_contributions_PC.set_index(['Production Steps'],inplace=True)
df_scenario_contributions_PC['Freshwater Ecotoxicity'] = (df_scenario_contributions_PC['Freshwater Ecotoxicity']/df_scenario_contributions_PC.at['LFP cell','Freshwater Ecotoxicity'])*100
df_scenario_contributions_PC['Global Warming Potential'] = (df_scenario_contributions_PC['Global Warming Potential']/df_scenario_contributions_PC.at['LFP cell','Global Warming Potential'])*100
df_scenario_contributions_PC['Human Carcinogen Toxicity'] = (df_scenario_contributions_PC['Human Carcinogen Toxicity']/df_scenario_contributions_PC.at['LFP cell','Human Carcinogen Toxicity'])*100
df_scenario_contributions_PC['Particulate Matter 2.5'] = (df_scenario_contributions_PC['Particulate Matter 2.5']/df_scenario_contributions_PC.at['LFP cell','Particulate Matter 2.5'])*100
df_scenario_contributions_PC['Terrestrial Ecotoxicity'] = (df_scenario_contributions_PC['Terrestrial Ecotoxicity']/df_scenario_contributions_PC.at['LFP cell','Terrestrial Ecotoxicity'])*100
# df_scenario_contributions_PC.rename(columns={'market for phosphoric acid, industrial grade, without water, in 85% solution state | phosphoric acid, industrial grade, without water, in 85% solution state | Cutoff, S - GLO':'Phosphoric acid','market for iron sulfate | iron sulfate | Cutoff, S - RoW':'Iron sulfate'},inplace=True)
# df_scenario_contributions_PC.reset_index(inplace=True)
# df_scenario_contributions_PC.at[df_scenario_contributions_PC['Production Steps'] == 'market for phosphoric acid, industrial grade, without water, in 85% solution state | phosphoric acid, industrial grade, without water, in 85% solution state | Cutoff, S - GLO','Production Steps'] = 'Phosphoric acid'
# df_scenario_contributions_PC.at[df_scenario_contributions_PC['Production Steps'] == 'market for iron sulfate | iron sulfate | Cutoff, S - RoW','Production Steps'] = 'Iron sulfate'


color_sequence = ['','#ffbe0b','#fb5607','#ff006e','#8338ec','#3a86ff','#f48c06','#FFD258','#FFE08B','#d9ed92','#bee7e8','#FFEDA5','#FC8952','#FF501E','#FC9A6B','lightgrey']
labels_sunburst=['LFP Cell','Cathode','Anode','Electrolyte','Separator','Cell container','Cathode current collector','Cathode paste','LFP Powder','Phosphoric acid','Iron sulfate','Lithium hydroxide','Anode paste','Anode current collector','Coated spherical graphite','other']
for x in df_scenario_contributions_PC.columns:

    fig = go.Figure(go.Sunburst(
        labels=labels_sunburst,
        parents=['','LFP Cell','LFP Cell','LFP Cell','LFP Cell','LFP Cell','Cathode','Cathode','Cathode paste','LFP Powder','LFP Powder','LFP Powder','Anode','Anode','Anode paste','LFP Cell'],
        values=[df_scenario_contributions_PC.at['LFP cell',x],
                df_scenario_contributions_PC.at['Cathode',x],
                df_scenario_contributions_PC.at['Anode',x],
                df_scenario_contributions_PC.at['Electrolyte',x],
                df_scenario_contributions_PC.at['Separator',x],
                df_scenario_contributions_PC.at['Cell container',x],
                df_scenario_contributions_PC.at['Cathode current collector',x],
                df_scenario_contributions_PC.at['Cathode paste',x],
                df_scenario_contributions_PC.at['Lithium iron phosphate powder',x],
                df_scenario_contributions_PC.at['market for phosphoric acid, industrial grade, without water, in 85% solution state | phosphoric acid, industrial grade, without water, in 85% solution state | Cutoff, S - GLO',x],
                df_scenario_contributions_PC.at['market for iron sulfate | iron sulfate | Cutoff, S - RoW',x],
                df_scenario_contributions_PC.at['Lithium hydroxide',x],
                df_scenario_contributions_PC.at['Anode paste',x],
                df_scenario_contributions_PC.at['Anode current collector',x],
                df_scenario_contributions_PC.at['Coated spherical graphite',x],
                (df_scenario_contributions_PC.at['LFP cell',x] - df_scenario_contributions_PC.at['Cathode',x] - df_scenario_contributions_PC.at['Anode',x] - df_scenario_contributions_PC.at['Electrolyte',x] - df_scenario_contributions_PC.at['Separator',x] - df_scenario_contributions_PC.at['Cell container',x])
        ],
        branchvalues="total",
        marker=dict(colors=color_sequence)
    ))
    fig.update_traces(textinfo="percent parent")
    fig.update_layout(showlegend=True,margin = dict(t=0, l=0, r=0, b=0),uniformtext=dict(minsize=15, mode='hide'))
    fig.show()
    fig.write_image("Data/Results/Figures/scenario_avg_sunburst_"+x+"_cell.png",format='png')



#### Bar chart by lithium source, system level

In [ ]:

cell_proc = ['BSS LFP','LFP module', 'LFP cell', 'Cathode', 'Cathode paste', 'Lithium iron phosphate powder', 'Lithium hydroxide', 'Anode', 'Anode paste', 'Coated spherical graphite', 'Electrolyte','Separator','Cell container','Housing','System controller','Inverter','Charger']


for x in df_scenario_contributions_lisource:
    df_cell_proc = x[x['Production Steps'].isin(cell_proc)]

    rest_cathode = []
    rest_anode = []
    rest_LFP = []
    rest_cell = []
    rest_module = []
    rest_system = []
    for i in df_cell_proc.columns[0:-1]:
        all_down_val_cathode = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Cathode',i].values[0]
        all_down_val_lfp = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Lithium iron phosphate powder',i].values[0]
        all_down_val_lioh = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Lithium hydroxide',i].values[0]
        all_down_val_anode = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Anode',i].values[0]
        all_down_val_csg = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Coated spherical graphite',i].values[0]
        all_down_val_cell = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'LFP cell',i].values[0]
        all_down_val_module = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'LFP module',i].values[0]
        val_inv = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Inverter',i].values[0]
        val_charger = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Charger',i].values[0]
        val_systcont = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'System controller',i].values[0]
        val_housing = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Housing',i].values[0]
        val_sep = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Separator',i].values[0]
        val_electro = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Electrolyte',i].values[0]
        val_cellcont = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Cell container',i].values[0]
        all_down_val_bss = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'BSS LFP',i].values[0]
        rest_cathode.append(all_down_val_cathode-all_down_val_lfp)
        rest_anode.append(all_down_val_anode-all_down_val_csg)
        rest_LFP.append(all_down_val_lfp-all_down_val_lioh)
        rest_cell.append(all_down_val_cell-all_down_val_anode-all_down_val_cathode-val_sep-val_electro-val_cellcont)
        rest_module.append(all_down_val_module-all_down_val_cell)
        rest_system.append(all_down_val_bss-val_inv-val_housing-val_charger-val_systcont-all_down_val_module)
    #index_val = df_scenario_contributions_lisource.index(x)
    rest_cathode.append("rest of cathode")
    rest_anode.append("rest of anode")
    rest_LFP.append("rest of LFP powder")
    rest_cell.append("rest of cell")
    rest_module.append("Components to modularize")
    rest_system.append("Other system components")
    x.loc[len(x)] = rest_cathode
    x.loc[len(x)] = rest_anode
    x.loc[len(x)] = rest_LFP
    x.loc[len(x)] = rest_cell
    x.loc[len(x)] = rest_module
    x.loc[len(x)] = rest_system


impacts_cell_proc = []

for i in df_scenario_contributions_lisource:
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    y5 = []
    y6 = []
    y7 = []
    y8 = []
    y9 = []
    y10 = []
    y11 = []
    y12 = []
    y13 = []
    y14 = []
    y15 = []
    y16 = []
    for x in i.columns[0:-1]:
        bss_value  = i.loc[i['Production Steps'] == 'BSS LFP',x].values[0]
        y1.append(i.loc[i['Production Steps'] == 'Lithium hydroxide',x].values[0]/bss_value)
        y2.append(i.loc[i['Production Steps'] == 'rest of LFP powder',x].values[0]/bss_value)
        y3.append(i.loc[i['Production Steps'] == 'rest of cathode',x].values[0]/bss_value)
        y4.append(i.loc[i['Production Steps'] == 'Coated spherical graphite',x].values[0]/bss_value)
        y5.append(i.loc[i['Production Steps'] == 'rest of anode',x].values[0]/bss_value)
        y6.append(i.loc[i['Production Steps'] == 'Electrolyte',x].values[0]/bss_value)
        y7.append(i.loc[i['Production Steps'] == 'Separator',x].values[0]/bss_value)
        y8.append(i.loc[i['Production Steps'] == 'Cell container',x].values[0]/bss_value)
        y9.append(i.loc[i['Production Steps'] == 'rest of cell',x].values[0]/bss_value)
        y10.append(i.loc[i['Production Steps'] == 'Components to modularize',x].values[0]/bss_value)
        y11.append(i.loc[i['Production Steps'] == 'Inverter',x].values[0]/bss_value)        
        y12.append(i.loc[i['Production Steps'] == 'Charger',x].values[0]/bss_value)        
        y13.append(i.loc[i['Production Steps'] == 'System controller',x].values[0]/bss_value)        
        y14.append(i.loc[i['Production Steps'] == 'Housing',x].values[0]/bss_value)        
        y15.append(i.loc[i['Production Steps'] == 'Other system components',x].values[0]/bss_value)        
        other = 1 - y1[-1] - y2[-1]- y3[-1] - y4[-1] - y5[-1] - y6[-1] - y7[-1] - y8[-1] - y9[-1] - y10[-1] - y11[-1] - y12[-1] - y13[-1] - y14[-1] - y15[-1]
        y16.append(other)
        impact_cell_proc = [y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16]
    impacts_cell_proc.append(impact_cell_proc)



for j in impacts_cell_proc:
    # create data
    x = ['Feshwater Ecotoxicity','Global Warming Potential','Human Carcinogen Toxicity','Particulate Matter, 2.5','Terrestrial Ecotoxicity']
    y1 = np.array(j[0])
    y2 = np.array(j[1])
    y3 = np.array(j[2])
    y4 = np.array(j[3])
    y5 = np.array(j[4])
    y6 = np.array(j[5])
    y7 = np.array(j[6])
    y8 = np.array(j[7])
    y9 = np.array(j[8])
    y10 = np.array(j[9])
    y11 = np.array(j[10])
    y12 = np.array(j[11])
    y13 = np.array(j[12])
    y14 = np.array(j[13])
    y15 = np.array(j[14])
    #y16 = np.array(j[15])

    # plot bars in stack manner
    plt.figure(figsize=(5,8))
    plt.bar(x, y1, color='#FFE08B',label="Lithium hydroxide")
    plt.bar(x, y2, bottom=y1, color='#FFD258',label="rest of LFP powder")
    plt.bar(x, y3, bottom=y1+y2, color='#ffbe0b',label="rest of Cathode")
    plt.bar(x, y4, bottom=y1+y2+y3, color='#FC9A6B',label="Coated spherical graphite")
    plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#fb5607',label="rest of Anode")
    plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#ff006e',label="Electrolyte")
    plt.bar(x, y7, bottom=y1+y2+y3+y4+y5+y6, color='#8338ec',label="Separator")
    plt.bar(x, y8, bottom=y1+y2+y3+y4+y5+y6+y7, color='#3a86ff',label="Cell container")
    plt.bar(x, y9, bottom=y1+y2+y3+y4+y5+y6+y7+y8, color='lightgrey',label="rest of Cell")
    plt.bar(x, y10, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9, color='#a98467',label="Components to Modularize")
    plt.bar(x, y11, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10, color='#38b000',label="Inverter")
    plt.bar(x, y12, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11, color='#006400',label="Charger")
    plt.bar(x, y13, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12, color='pink',label="System controller")
    plt.bar(x, y14, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13, color='purple',label="Housing")
    plt.bar(x, y15, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13+y14, color='#5c677d',label="Other BSS Components")
    #plt.bar(x, y16, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13+y14+y15, color='lavender',label="Other")



    plt.legend(loc='best',bbox_to_anchor=(0.5, -0.05))
    plt.ylim(0,1)
    #plt.xticks(rotation=45)
    plt.savefig('Data/Results/Figures/BSS_process_breakdown_'+str(impacts_cell_proc.index(j))+'.png') # order of impacts 'Freshwater Ecotoxicity', 'Global Warming Potential', 'Human Carcinogen Toxicity', 'Particulate Matter 2.5', 'Terrestrial Ecotoxicity',
    plt.show()

### PRC LCIA data

In [ ]:
df = pd.read_excel('Data/Results/BSS_LFP_CN.xlsx',sheet_name="Impacts",skiprows=1)

## PRC Analysis

Stats for all impact categories

In [ ]:
list_impact = df['Result'].to_list()

impact_names = df['Impact category'].to_list()
impact_units = df['Reference unit'].to_list()

df_impact_stats_cn = pd.DataFrame([impact_names, impact_units, list_impact])

df_impact_stats_cn = df_impact_stats_cn.transpose()

df_impact_stats_cn.rename(columns={0:'Impacts',1:'Units',2:'Values'},inplace=True)

#### Contribution trees

In [ ]:
steps_names_dict = {}

for x in steps_manuf:
    steps_names_dict.update({steps_manuf.index(x):x})

Particulate matter

In [ ]:
df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_CN_PM.xlsx',skiprows=1)
df['DownstreamProcess'] = df[df.columns[0:-1]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)


bss_name = 'BSS LFP_CN'
bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg PM2.5 eq]'].values[0]
scenario_process_contributions = []
for j in steps_manuf:
    step_name = j + '_CN'
    step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg PM2.5 eq]'].values[0]
    ratio = step_val/bss_total
    scenario_process_contributions.append(ratio)
process_contributions_PM.append(scenario_process_contributions)

Global warming potential

In [ ]:
df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_CN_GWP.xlsx',skiprows=1)
df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
lambda x: ','.join(x.dropna().astype(str)),
axis=1
)


bss_name = 'BSS LFP_CN'
bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg CO2 eq]'].values[0]
scenario_process_contributions = []
for j in steps_manuf:
    step_name = j + '_CN'
    step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg CO2 eq]'].values[0]
    ratio = step_val/bss_total
    scenario_process_contributions.append(ratio)
process_contributions_PM.append(scenario_process_contributions)

Freshwater ecotoxicity

In [ ]:
df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_CN_FET.xlsx',skiprows=1)
df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
lambda x: ','.join(x.dropna().astype(str)),
axis=1
)


bss_name = 'BSS LFP_CN'
bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
scenario_process_contributions = []
for j in steps_manuf:
    step_name = j + '_CN'
    step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
    ratio = step_val/bss_total
    scenario_process_contributions.append(ratio)
process_contributions_PM.append(scenario_process_contributions)

Human carcinogen toxicity

In [ ]:
df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_CN_HTP.xlsx',skiprows=1)
df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
lambda x: ','.join(x.dropna().astype(str)),
axis=1
)


bss_name = 'BSS LFP_CN'
bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
scenario_process_contributions = []
for j in steps_manuf:
    step_name = j + '_CN'
    step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
    ratio = step_val/bss_total
    scenario_process_contributions.append(ratio)
process_contributions_PM.append(scenario_process_contributions)


Terrestrial ecotoxicity

In [ ]:
df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_CN_TET.xlsx',skiprows=1)
df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
lambda x: ','.join(x.dropna().astype(str)),
axis=1
)


bss_name = 'BSS LFP_CN'
bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [kg 1,4-DCB]'].values[0]
scenario_process_contributions = []
for j in steps_manuf:
    step_name = j + '_CN'
    step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [kg 1,4-DCB]'].values[0]
    ratio = step_val/bss_total
    scenario_process_contributions.append(ratio)
process_contributions_PM.append(scenario_process_contributions)

### Process contributions towards the five impact categories

In [ ]:
diff_contributions = [process_contributions_FET,process_contributions_GWP,process_contributions_HCT,process_contributions_PM,process_contributions_TET]

lists_scenario_contributions = []
for x in diff_contributions:
    df = pd.DataFrame(x)
    df.rename(columns=steps_names_dict,inplace=True)
    diff_cont_scenario_averages = df.mean(axis=0).to_list()
    lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions = df_scenario_contributions.transpose()

df_scenario_contributions.rename(columns={0:'Freshwater Ecotoxicity',1:'Global Warming Potential',2:'Human Carcinogen Toxicity',3:'Particulate Matter 2.5',4:'Terrestrial Ecotoxicity'},inplace=True)

df_scenario_contributions["Production Steps"] = steps_manuf

### Electricity impacts

In [ ]:
df_elec = pd.read_excel('Data/Results/BSS_LFP_CN.xlsx',sheet_name="Process impact contributions",skiprows=2)
df_elec = df_elec.loc[2:,]
df_elec.drop(columns = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 3", "Process"],inplace=True)
df_elec.rename(columns={'Unnamed: 2':'Impact Category'},inplace=True)
df_elec.set_index(df_elec.columns[0],inplace=True)
df_elec = df_elec.transpose()
df_elec.reset_index(inplace=True)
df_elec = df_elec[df_elec['index'].str.contains('electricity')]
df_elec.drop(columns=['index'],inplace=True)
df_elec = pd.DataFrame(df_elec.sum())

# electricity used in the process, exculding electricity used to produce the materials being used
dfs_elec_impacts_cn = df_elec.copy()
dfs_elec_impacts_cn['scenario_avg_total_impact'] = df_impact_stats_cn['Values'].to_list()
dfs_elec_impacts_cn['ratio_elec_all_impacts'] = dfs_elec_impacts_cn[0]/dfs_elec_impacts_cn['scenario_avg_total_impact']*100

### Transportation impacts

In [ ]:
df_trans = pd.read_excel('Data/Results/BSS_LFP_CN.xlsx',sheet_name="Process impact contributions",skiprows=2)
df_trans = df_trans.loc[2:,]
df_trans.drop(columns = ["Unnamed: 0", "Unnamed: 1", "Unnamed: 3", "Process"],inplace=True)
df_trans.rename(columns={'Unnamed: 2':'Impact Category'},inplace=True)
df_trans.set_index(df_trans.columns[0],inplace=True)
df_trans = df_trans.transpose()
df_trans.reset_index(inplace=True)
df_trans = df_trans[df_trans['index'].str.contains('transport')]
df_trans.drop(columns=['index'],inplace=True)
df_trans = pd.DataFrame(df_trans.sum())

# transportation used in the process, exculding transportation of non-lithium and non-graphite materials being used
dfs_transport_impacts_cn = df_trans.copy()
dfs_transport_impacts_cn['scenario_avg_total_impact'] = df_impact_stats_cn['Values'].to_list()
dfs_transport_impacts_cn['ratio_transport_all_impacts'] = dfs_transport_impacts_cn[0]/dfs_transport_impacts_cn['scenario_avg_total_impact']*100


## Figures US and PRC

Differences between all US scenarios and PRC results

In [ ]:
df_impacts_US_China = df_impact_stats.copy()
df_impacts_US_China.rename(columns={'median':'USimpacts'},inplace=True)
df_impacts_US_China['PRC'] = df_impact_stats_cn['Values'].to_list()
df_impacts_US_China['PRC-US'] = df_impacts_US_China['PRC'] - df_impacts_US_China['USimpacts']
df_impacts_US_China['PRC-US Perc'] = (df_impacts_US_China['PRC-US']/df_impacts_US_China['USimpacts'])*100


df_impacts_US_China['color'] = ['#3a86ff' if float( 
    x) < 0 else '#fb5607' for x in df_impacts_US_China['PRC-US Perc']]

# # Sort values from lowest to highest 
# df.sort_values('car_sales_z', inplace=True) 
  
# # Resets initial index in Dataframe to None 
# df.reset_index(inplace=True) 

plt.rcParams.update({'font.size': 18,'font.family':'Arial'})
# Draw plot 
plt.figure(figsize=(14, 10), dpi=80) 
  
# Plotting the horizontal lines 
plt.hlines(y=df_impacts_US_China['Impacts'], xmin=0, xmax=df_impacts_US_China['PRC-US Perc'],
           color=df_impacts_US_China.color, alpha=0.8, linewidth=18) 
    
# Optional grid layout 
plt.grid(linestyle='--', alpha=0.5) 
  
# Displaying the Diverging Bar Chart 
plt.savefig('Data/Results/Figures/US-PRC_Diff_Plot.png',dpi=600)

Differences between US spodumene scenarios and PRC results

In [ ]:
df_impacts_US_China = df_diff_scenarios_impacts[1][['Impacts','Units','mean']]
df_impacts_US_China.rename(columns={'mean':'USimpacts'},inplace=True)
df_impacts_US_China['PRC'] = df_impact_stats_cn['Values'].to_list()
df_impacts_US_China['PRC-US'] = df_impacts_US_China['PRC'] - df_impacts_US_China['USimpacts']
df_impacts_US_China['PRC-US Perc'] = (df_impacts_US_China['PRC-US']/df_impacts_US_China['USimpacts'])*100


df_impacts_US_China['color'] = ['#006d77' if float( 
    x) < 0 else '#b56576' for x in df_impacts_US_China['PRC-US Perc']]

# # Sort values from lowest to highest 
# df.sort_values('car_sales_z', inplace=True) 
  
# # Resets initial index in Dataframe to None 
# df.reset_index(inplace=True) 

plt.rcParams.update({'font.size': 18,'font.family':'Arial'})
# Draw plot 
plt.figure(figsize=(14, 10), dpi=80) 
  
# Plotting the horizontal lines 
plt.hlines(y=df_impacts_US_China['Impacts'], xmin=0, xmax=df_impacts_US_China['PRC-US Perc'],
           color=df_impacts_US_China.color, alpha=0.8, linewidth=18) 
    
# Optional grid layout 
plt.grid(linestyle='--', alpha=0.5) 
  
# Displaying the Diverging Bar Chart 
plt.savefig('Data/Results/Figures/USSpodumene-PRC_Diff_Plot.png',dpi=600)

### Monte Carlo analysis to get range of sum of all impact categories from US manufacturing in year 2030 for example

In [ ]:
# estimate of environmental impacts associated with manufacturing in 2030
mean = (450+620)/2 # forecast from McKinsey and Company, lower and upper bound for utility-scale instillations, GW
stdev = (mean-450)/3   # 99.73% chance the sample will fall in desired range of normal dist

values_2030_cap = [random.gauss(mean, stdev) for _ in range(5000)]
values_manuf_perc = [random.choice([0.50,0.75,0.90]) for _ in range(5000)] # uniform distribution
values_lfp_perc = [random.choice([0.70,0.87,0.92]) for _ in range(5000)] # uniform distribution, https://www.fastmarkets.com/insights/growing-lfp-adoption-drives-need-for-more-transparency-across-chemistrys-supply-chain/#:~:text=Rapid%20growth%20of%20ESS%20supports%20LFP%20demand&text=It%27s%20affordable%20and%20stable%2C%20and,batteries%2C%20O%27Hara%20said.

values_2030_US_manuf_cap = [a * b for a,b in zip(values_2030_cap, values_manuf_perc)]
values_2030_US_manuf_cap = [a * b for a,b in zip(values_2030_US_manuf_cap,values_lfp_perc)]

impact_random = []
for i,r in df_impact_stats.iterrows():
    impact_random_values = [random.gauss(r['mean'],r['standard deviation']) for _ in range(5000)]
    values_total_impacts_2030 = [a * b * 1000000 for a, b in zip(values_2030_US_manuf_cap, impact_random_values)]
    impact_random.append(values_total_impacts_2030)


avg_total_impact_2030 = []
for x in impact_random:
    impacts_avg = sum(x)/len(x)
    avg_total_impact_2030.append(impacts_avg)

avg_total_impact_2030_tonne = [x/1000 for x in avg_total_impact_2030]

### Water consumption mini analysis

In [ ]:
dfs_WCP = []

for x in range(0,12,1):
    df = pd.read_excel('Data/Results/contribution_trees/contribution_tree_'+str(x)+'_WCP.xlsx',skiprows=1)
    df['DownstreamProcess'] = df[df.columns[0:-1]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    dfs_WCP.append(df)


process_contributions_WCP = []
for x in range(0,12,1):
    df = dfs_WCP[x]
    bss_name = 'BSS LFP_' + str(x)
    bss_total = df.loc[df['DownstreamProcess'] == bss_name, 'Result [m3]'].values[0]
    scenario_process_contributions = []
    for j in steps_manuf:
        step_name = j + '_' + str(x)
        step_val = df.loc[df['DownstreamProcess'] == step_name, 'Result [m3]'].values[0]
        ratio = step_val/bss_total
        scenario_process_contributions.append(ratio)
    process_contributions_WCP.append(scenario_process_contributions)

lists_scenario_contributions = []
df = pd.DataFrame(process_contributions_WCP)
df.rename(columns=steps_names_dict,inplace=True)
diff_cont_scenario_averages = df.mean(axis=0).to_list()
lists_scenario_contributions.append(diff_cont_scenario_averages)

df_scenario_contributions_contbrine = pd.DataFrame(lists_scenario_contributions)
df_scenario_contributions_contbrine = df_scenario_contributions_contbrine.transpose()

df_scenario_contributions_contbrine["Production Steps"] = steps_manuf



cell_proc = ['BSS LFP','LFP module', 'LFP cell', 'Cathode', 'Cathode paste', 'Lithium iron phosphate powder', 'Lithium hydroxide', 'Anode', 'Anode paste', 'Coated spherical graphite', 'Electrolyte','Separator','Cell container','Housing','System controller','Inverter','Charger']

df_scenario_contributions_lisource = [df_scenario_contributions_contbrine]
for x in df_scenario_contributions_lisource:
    df_cell_proc = x[x['Production Steps'].isin(cell_proc)]

    rest_cathode = []
    rest_anode = []
    rest_LFP = []
    rest_cell = []
    rest_module = []
    rest_system = []
    for i in df_cell_proc.columns[0:-1]:
        all_down_val_cathode = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Cathode',i].values[0]
        all_down_val_lfp = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Lithium iron phosphate powder',i].values[0]
        all_down_val_lioh = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Lithium hydroxide',i].values[0]
        all_down_val_anode = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Anode',i].values[0]
        all_down_val_csg = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Coated spherical graphite',i].values[0]
        all_down_val_cell = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'LFP cell',i].values[0]
        all_down_val_module = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'LFP module',i].values[0]
        val_inv = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Inverter',i].values[0]
        val_charger = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Charger',i].values[0]
        val_systcont = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'System controller',i].values[0]
        val_housing = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Housing',i].values[0]
        val_sep = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Separator',i].values[0]
        val_electro = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Electrolyte',i].values[0]
        val_cellcont = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'Cell container',i].values[0]
        all_down_val_bss = df_cell_proc.loc[df_cell_proc['Production Steps'] == 'BSS LFP',i].values[0]
        rest_cathode.append(all_down_val_cathode-all_down_val_lfp)
        rest_anode.append(all_down_val_anode-all_down_val_csg)
        rest_LFP.append(all_down_val_lfp-all_down_val_lioh)
        rest_cell.append(all_down_val_cell-all_down_val_anode-all_down_val_cathode-val_sep-val_electro-val_cellcont)
        rest_module.append(all_down_val_module-all_down_val_cell)
        rest_system.append(all_down_val_bss-val_inv-val_housing-val_charger-val_systcont-all_down_val_module)
    #index_val = df_scenario_contributions_lisource.index(x)
    rest_cathode.append("rest of cathode")
    rest_anode.append("rest of anode")
    rest_LFP.append("rest of LFP powder")
    rest_cell.append("rest of cell")
    rest_module.append("Components to modularize")
    rest_system.append("Other system components")
    x.loc[len(x)] = rest_cathode
    x.loc[len(x)] = rest_anode
    x.loc[len(x)] = rest_LFP
    x.loc[len(x)] = rest_cell
    x.loc[len(x)] = rest_module
    x.loc[len(x)] = rest_system



# importing package

impacts_cell_proc = []

for i in df_scenario_contributions_lisource:
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    y5 = []
    y6 = []
    y7 = []
    y8 = []
    y9 = []
    y10 = []
    y11 = []
    y12 = []
    y13 = []
    y14 = []
    y15 = []
    y16 = []
    for x in i.columns[0:-1]:
        bss_value  = i.loc[i['Production Steps'] == 'BSS LFP',x].values[0]
        y1.append(i.loc[i['Production Steps'] == 'Lithium hydroxide',x].values[0]/bss_value)
        y2.append(i.loc[i['Production Steps'] == 'rest of LFP powder',x].values[0]/bss_value)
        y3.append(i.loc[i['Production Steps'] == 'rest of cathode',x].values[0]/bss_value)
        y4.append(i.loc[i['Production Steps'] == 'Coated spherical graphite',x].values[0]/bss_value)
        y5.append(i.loc[i['Production Steps'] == 'rest of anode',x].values[0]/bss_value)
        y6.append(i.loc[i['Production Steps'] == 'Electrolyte',x].values[0]/bss_value)
        y7.append(i.loc[i['Production Steps'] == 'Separator',x].values[0]/bss_value)
        y8.append(i.loc[i['Production Steps'] == 'Cell container',x].values[0]/bss_value)
        y9.append(i.loc[i['Production Steps'] == 'rest of cell',x].values[0]/bss_value)
        y10.append(i.loc[i['Production Steps'] == 'Components to modularize',x].values[0]/bss_value)
        y11.append(i.loc[i['Production Steps'] == 'Inverter',x].values[0]/bss_value)        
        y12.append(i.loc[i['Production Steps'] == 'Charger',x].values[0]/bss_value)        
        y13.append(i.loc[i['Production Steps'] == 'System controller',x].values[0]/bss_value)        
        y14.append(i.loc[i['Production Steps'] == 'Housing',x].values[0]/bss_value)        
        y15.append(i.loc[i['Production Steps'] == 'Other system components',x].values[0]/bss_value)        
        other = 1 - y1[-1] - y2[-1]- y3[-1] - y4[-1] - y5[-1] - y6[-1] - y7[-1] - y8[-1] - y9[-1] - y10[-1] - y11[-1] - y12[-1] - y13[-1] - y14[-1] - y15[-1]
        y16.append(other)
        impact_cell_proc = [y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16]
    impacts_cell_proc.append(impact_cell_proc)



for j in impacts_cell_proc:
    # create data
    x = ['WCP']
    y1 = np.array(j[0])
    y2 = np.array(j[1])
    y3 = np.array(j[2])
    y4 = np.array(j[3])
    y5 = np.array(j[4])
    y6 = np.array(j[5])
    y7 = np.array(j[6])
    y8 = np.array(j[7])
    y9 = np.array(j[8])
    y10 = np.array(j[9])
    y11 = np.array(j[10])
    y12 = np.array(j[11])
    y13 = np.array(j[12])
    y14 = np.array(j[13])
    y15 = np.array(j[14])
    #y16 = np.array(j[15])

    # plot bars in stack manner
    plt.figure(figsize=(5,8))
    plt.bar(x, y1, color='#FFE08B',label="Lithium hydroxide")
    plt.bar(x, y2, bottom=y1, color='#FFD258',label="rest of LFP powder")
    plt.bar(x, y3, bottom=y1+y2, color='#ffbe0b',label="rest of Cathode")
    plt.bar(x, y4, bottom=y1+y2+y3, color='#FC9A6B',label="Coated spherical graphite")
    plt.bar(x, y5, bottom=y1+y2+y3+y4, color='#fb5607',label="rest of Anode")
    plt.bar(x, y6, bottom=y1+y2+y3+y4+y5, color='#ff006e',label="Electrolyte")
    plt.bar(x, y7, bottom=y1+y2+y3+y4+y5+y6, color='#8338ec',label="Separator")
    plt.bar(x, y8, bottom=y1+y2+y3+y4+y5+y6+y7, color='#3a86ff',label="Cell container")
    plt.bar(x, y9, bottom=y1+y2+y3+y4+y5+y6+y7+y8, color='lightgrey',label="rest of Cell")
    plt.bar(x, y10, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9, color='#a98467',label="Components to Modularize")
    plt.bar(x, y11, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10, color='#38b000',label="Inverter")
    plt.bar(x, y12, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11, color='#006400',label="Charger")
    plt.bar(x, y13, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12, color='pink',label="System controller")
    plt.bar(x, y14, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13, color='purple',label="Housing")
    plt.bar(x, y15, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13+y14, color='#5c677d',label="Other BSS Components")
    #plt.bar(x, y16, bottom=y1+y2+y3+y4+y5+y6+y7+y8+y9+y10+y11+y12+y13+y14+y15, color='lavender',label="Other")



    plt.legend(loc='best',bbox_to_anchor=(0.5, -0.05))
    plt.ylim(0,1)
    #plt.xticks(rotation=45)
    #plt.savefig('Data/Results/Figures/BSS_process_breakdown_'+str(impacts_cell_proc.index(j))+'.png') # order of impacts 'Freshwater Ecotoxicity', 'Global Warming Potential', 'Human Carcinogen Toxicity', 'Particulate Matter 2.5', 'Terrestrial Ecotoxicity',
    plt.show()